In [1]:
import subprocess
import os
import gzip
import shutil

# Description

We use convbin app from RTKLIB.
Clone https://github.com/tomojitakasu/RTKLIB/tree/rtklib_2.4.3 and run \
$ RTKLIB/app/convbin/gcc make

This notebook runs the following process:
1. Decompress the ubx files
2. Convert ubx files to obs files using convbin
3. Compress the obs files

## Directry structure

Our ubx files are compressed and stored in "SA_ubx_data" folder.
 
> SA_ubx_data/
>> rov_202304010000.log.ubx.gz\
>> rov_202304020000.log.ubx.gz\
>> rov_202304030000.log.ubx.gz, ...

Converted obs files are compressed and stored in "SA_GNSS_data/22300000_obs/" folder.
 
 > 22300000_obs/
 >> 202304/
 >>> rov_20230401.obs.gz\
 >>> rov_20230402.obs.gz, ...

## Minor bugs
convbin has options to limit the time range of output obs files, -ts and -te.\
They are declared as following:\
-ts 2023/04/01 10:00:00\
-te 2023/04/01 12:00:00.\
However, -ts recognizes date but doesn't recognize time.\
On top of that, -te does nothing.


In [2]:
def convert_ubx(input_file, output_file, start_time, end_time, convbin_path="RTKLIB-rtklib_2.4.3/app/consapp/convbin/gcc/convbin"):
    options = [
        "-ts", start_time,
        "-te", end_time,
        "-ha", "00000000/TWIVSP6037L", # antenna type
        "-f", "5", # number of frequencies
        "-d", output_file
    ]

    cmd = [convbin_path, *options, input_file]

    result = subprocess.run(cmd, capture_output=True, text=True)

    if result.returncode == 0:
        print(input_file+" has been successfully converted.")
        print(result.stdout)
    else:
        print("Failed to process "+input_file)
        print(result.stderr)

In [22]:
# get filelist
gz_path = "SA_ubx_data"
list_ubx = [d for d in os.listdir(gz_path) if d.find('rov_202304')==0]
list_ubx.sort()
list_ubx

['rov_202304010000.log.ubx.gz',
 'rov_202304020000.log.ubx.gz',
 'rov_202304030000.log.ubx.gz',
 'rov_202304031214.log.ubx.gz',
 'rov_202304040000.log.ubx.gz',
 'rov_202304050000.log.ubx.gz',
 'rov_202304060000.log.ubx.gz',
 'rov_202304070000.log.ubx.gz',
 'rov_202304080000.log.ubx.gz',
 'rov_202304090000.log.ubx.gz',
 'rov_202304090950.log.ubx.gz',
 'rov_202304100000.log.ubx.gz',
 'rov_202304110000.log.ubx.gz',
 'rov_202304120000.log.ubx.gz',
 'rov_202304130000.log.ubx.gz',
 'rov_202304140000.log.ubx.gz',
 'rov_202304150000.log.ubx.gz',
 'rov_202304160000.log.ubx.gz',
 'rov_202304170000.log.ubx.gz',
 'rov_202304210252.log.ubx.gz',
 'rov_202304220000.log.ubx.gz',
 'rov_202304230000.log.ubx.gz',
 'rov_202304240000.log.ubx.gz',
 'rov_202304250000.log.ubx.gz',
 'rov_202304260000.log.ubx.gz',
 'rov_202304270000.log.ubx.gz',
 'rov_202304280000.log.ubx.gz',
 'rov_202304290000.log.ubx.gz',
 'rov_202304291442.log.ubx.gz',
 'rov_202304300000.log.ubx.gz']

In [23]:
for i, gz_file in enumerate(list_ubx):
    # decompress the gz file
    obs_path = "SA_GNSS_data/22300000_obs/"+gz_file[4:10] # directry to store obs file
    if not os.path.exists(obs_path):
        os.makedirs(obs_path)
    gz_file_dir = os.path.join(gz_path, gz_file)
    decompressed_file_dir = os.path.join(obs_path, gz_file[:-3])
    with gzip.open(gz_file_dir, mode="rb") as gzip_file:
        with open(decompressed_file_dir, mode="wb") as decompressed_file:
            shutil.copyfileobj(gzip_file, decompressed_file)
    # convert to obs file
    input_file = decompressed_file_dir
    year = gz_file[4:8]
    month = gz_file[8:10]
    day = gz_file[10:12]
    start_time =year+"/"+month+"/"+day+" 00:00:00"
    end_time =year+"/"+month+"/"+day+" 23:59:59"
    output_folder = os.path.join(obs_path,"rov_"+year+month+day)
    convert_ubx(input_file, output_folder, start_time, end_time)
    # pick up obs file from the output
    obs_file = os.path.join(output_folder, gz_file[:-6]+"obs")
    new_path = shutil.move(obs_file, obs_path)
    shutil.rmtree(output_folder)
    obs_file = os.path.join(obs_path, gz_file[:-6]+"obs")
    
    # compress the obs file
    with open(obs_file, "rb") as obs_raw:
        with gzip.open(os.path.join(obs_path,"rov_"+year+month+day+".obs.gz"), "wb") as obs_compressed:
            shutil.copyfileobj(obs_raw, obs_compressed)
    # remove raw obs file
    os.remove(obs_file)
    # delete decompressed ubx file
    os.remove(decompressed_file_dir)



SA_GNSS_data/22300000_obs/202304/rov_202304010000.log.ubx has been successfully converted.

SA_GNSS_data/22300000_obs/202304/rov_202304020000.log.ubx has been successfully converted.

SA_GNSS_data/22300000_obs/202304/rov_202304030000.log.ubx has been successfully converted.

SA_GNSS_data/22300000_obs/202304/rov_202304031214.log.ubx has been successfully converted.

SA_GNSS_data/22300000_obs/202304/rov_202304040000.log.ubx has been successfully converted.

SA_GNSS_data/22300000_obs/202304/rov_202304050000.log.ubx has been successfully converted.

SA_GNSS_data/22300000_obs/202304/rov_202304060000.log.ubx has been successfully converted.

SA_GNSS_data/22300000_obs/202304/rov_202304070000.log.ubx has been successfully converted.

SA_GNSS_data/22300000_obs/202304/rov_202304080000.log.ubx has been successfully converted.

